## EY Data Challenge - Landsat Land Surface Temperature

This sample notebook can be used to create a Landsat Land Surface Temperature (LST) product. The notebook creates a cloud-filtered median mosaic for any time period and location and then creates the LST product. A median mosaic reflects the "median" value of pixels for all spectral bands in the time series. When scenes within a time series contain clouds, the use of a median calculation can statistically remove clouds from the final median mosaic product, assuming there are plenty of clear pixels within the time series. The baseline data is [Landsat Collection-2 Level-2](https://www.usgs.gov/landsat-missions/landsat-collection-2) data from the MS Planetary Computer catalog.

In [ ]:
!pip install planetary-computer odc.stac pystac_client stackstac rioxarray

  Using cached odc_stac-0.3.11-py3-none-any.whl.metadata (5.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import jet,RdYlGn

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

### Discover and load the data for analysis

First, we define our area of interest using latitude and longitude coordinates.

In [ ]:
# Define the bounding box for the entire data region using (Latitude, Longitude)
# This is the region of New York City that contains our temperature dataset
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

In [ ]:
# Calculate the bounds for doing an archive data search
# bounds = (min_lon, min_lat, max_lon, max_lat)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

In [ ]:
# Define the time window
# We will use a period of 3 months to search for data
time_window = "2021-06-01/2021-09-01"

Using the `pystac_client` we can search the Planetary Computer's STAC endpoint for items matching our query parameters. The query searches for "low cloud" scenes with overall cloud cover <20%. We will also limit our search to Landsat-8 to avoid the Landsat-7 scan line corrector failure. The result is the number of scenes matching our search criteria that touch our area of interest. Some of these may be partial scenes or contain clouds.

In [ ]:
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["landsat-8-c2-l2"],
    query={"eo:cloud_cover": {"lt": 20}},
)

In [ ]:
items = list(search.get_items())
print('This is the number of scenes that touch our region:',len(items))

This is the number of scenes that touch our region: 5


Next, we'll load the data into an [xarray](https://xarray.pydata.org/en/stable/) DataArray using [stackstac](https://stackstac.readthedocs.io/). We will only keep the commonly used spectral bands (Red, Green, Blue, NIR, Surface Temperature). There are also several other <b>important settings for the data</b>: We have changed the projection to epsg=4326 which is standard latitude-longitude in degrees. We have specified the spatial resolution of each pixel to be 30-meters.

In [ ]:
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [ ]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees
resolution = 30  # meters per pixel
scale = resolution / 111320.0 # degrees per pixel for crs=4326

### Landsat Band Summary
The following list of bands will be loaded by the Open Data Cube (ODC) stac command:<br>
We will use two load commands to separate the RGB data for the Surface Temperature data.<br><br>

Surface Temperature Bands:<br>
Band 10 = Surface Temperature = ST_B10 = 100m<br>
Band 11 = Thermal Infrared = lwir11 = 100m<br>

Derived Atmospheric and QA Data:<br>
ST_ATRAN = Atmospheric Transmittance (Band 10) = Derived<br>
ST_CDIST = Cloud Distance = Derived<br>
ST_DRAD = Downwelling Radiation (Band 10) = Derived<br>
ST_QA = Quality Assessment = Derived<br>
ST_EMIS = Surface Emissivity = Derived<br>

In [ ]:
data1 = stac_load(
    items,
    bands=["ST_B10", "ST_ATRAN", "ST_CDIST", "ST_DRAD", "ST_QA", "ST_EMIS"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

In [ ]:
data2 = stac_load(
    items,
    bands=["lwir11"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

Since the data matching our query isn't too large we can persist it in distributed memory. Once in memory, subsequent operations will be much faster.

In [ ]:
# View the dimensions of our XARRAY and the loaded variables
# This insures we have the right coordinates and spectral bands in our xarray
display(data1)

<xarray.Dataset> Size: 16MB
Dimensions:      (latitude: 484, longitude: 558, time: 5)
Coordinates:
  * latitude     (latitude) float64 4kB 40.88 40.88 40.88 ... 40.75 40.75 40.75
  * longitude    (longitude) float64 4kB -74.01 -74.01 -74.01 ... -73.86 -73.86
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 40B 2021-06-07T15:39:15.904901 ... 202...
Data variables:
    ST_B10       (time, latitude, longitude) uint16 3MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    ST_ATRAN     (time, latitude, longitude) uint16 3MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    ST_CDIST     (time, latitude, longitude) uint16 3MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    ST_DRAD      (time, latitude, longitude) uint16 3MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    ST_QA        (time, latitude, longitude) uint16 3MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>
    ST_EMIS      (time, latitude, longitude) uint16 3MB dask.array<chunksize=(1, 484, 558), meta=np.ndarray>

In [ ]:
# Persist the data in memory for faster operations
data1 = data1.persist()

### Save the output data in a GeoTIFF file

In [ ]:
scene = 0

In [ ]:
# Only select one of the time slices to output
data3 = data1.isel(time=scene)

In [ ]:
filename = "Landsat_LST.tiff"

In [ ]:
# Calculate the dimensions of the file
height = data3.dims["latitude"]
width = data3.dims["longitude"]

In [ ]:
# Define the Coordinate Reference System (CRS) to be common Lat-Lon coordinates
# Define the tranformation using our bounding box so the Lat-Lon information is written to the GeoTIFF
gt = rasterio.transform.from_bounds(lower_left[1],lower_left[0],upper_right[1],upper_right[0],width,height)
data3.rio.write_crs("epsg:4326", inplace=True)
data3.rio.write_transform(transform=gt, inplace=True);

In [ ]:
# Create the GeoTIFF output file using the defined parameters
with rasterio.open(filename,'w',driver='GTiff',width=width,height=height,
                   crs='epsg:4326',transform=gt,count=6,compress='lzw',dtype='float64') as dst:
    dst.write(data3.ST_B10.values,1)
    dst.write(data3.ST_ATRAN.values,2)
    dst.write(data3.ST_CDIST.values,3)
    dst.write(data3.ST_DRAD.values, 4)
    dst.write(data3.ST_QA.values, 5)
    dst.write(data3.ST_EMIS.values, 6)
    dst.close()

In [ ]:
# Show the new saved output file
!ls *.tiff

Landsat_LST.tiff
